In [113]:
import os
from deta import Deta
from dotenv import load_dotenv
import sqlite3
import pandas as pd
import numpy as np

### Loading DB key

In [2]:
load_dotenv(".env")
DETA_KEY = os.getenv("DETA_KEY")

### Initializing DB key

In [3]:
deta = Deta(DETA_KEY)

### Initializing DB

In [4]:
db = deta.Base("exp_tracker_app")

### DB functions

In [5]:
def insert_period(period, incomes, expenses, comment):
    """Returns monthly report"""
    return db.put({"key": period, "incomes": incomes, "expenses": expenses, "comment": comment})

def fetch_all_periods():
    """Returns a dict of all periods"""
    res = db.fetch()
    return res.items

def get_period(period):
    return db.get(period)

### Example data

In [6]:
period = "2022_May"
comment = "init"
incomes = {'Salary': 1500, 'Dividend': 500, 'Invest Return': 100}
expenses = {'Food': 200, 'Loan': 300, 'CC Bill': 30}

#### DB inserting example data

In [7]:
insert_period(period, incomes, expenses, comment)

{'comment': 'init',
 'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
 'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
 'key': '2022_May'}

#### Fetching all data from DB

In [8]:
fetch_all_periods()

[{'comment': 'init',
  'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
  'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
  'key': '2022_May'},
 {'comment': 'first entry',
  'expenses': {'CC Bill': 30, 'Food': 40, 'Loan': 170, 'Others': 200},
  'incomes': {'Dividend': 10, 'Invest Return': 20, 'Salary': 450},
  'key': '2023_January'}]

#### Fetching single period data from DB

In [9]:
get_period("2022_May")

{'comment': 'init',
 'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
 'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
 'key': '2022_May'}

In [179]:
db2 = deta.Base("wallet_db")

In [185]:
def insert_trans(period, date, _type, amount, wallet_id, category_id, transfer_wallet_id, trans_amount, subcategory_id, note):
    """Inserts daily transaction"""
    return db2.put({'key': period, 'date': date, 'type': _type, 'amount': amount, 'wallet_id': wallet_id, 'category_id': category_id, 'transfer_wallet_id': transfer_wallet_id, 'trans_amount': trans_amount, 'subcategory_id': subcategory_id, 'note': note})

def fetch_all_trans():
    """Returns a dict of all periods"""
    res = db2.fetch()
    return res.items

def get_period(period):
    """Returns monthly report"""
    return db2.get(period)

In [164]:
insert_trans('Jul-23', '24/06/2023', 'inc', 10, 1, 2, -1, 0, 0, 'test')

{'amount': 10,
 'category_id': 2,
 'date': '24/06/2023',
 'key': 'Jul-23',
 'note': 'test',
 'subcategory_id': 0,
 'trans_amount': 0,
 'transfer_wallet_id': -1,
 'type': 'inc',
 'wallet_id': 1}

In [180]:
fetch_all_trans()

[{'amount': -149,
  'category_id': 11,
  'date': '2020-06-24',
  'key': '1592976346122',
  'mon_year': 'Jun_2020',
  'note': 'mou belt',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 3},
 {'amount': -299,
  'category_id': 11,
  'date': '2020-06-24',
  'key': '1592976705246',
  'mon_year': 'Jun_2020',
  'note': 'wedding shoe',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 3},
 {'amount': -1500,
  'category_id': 11,
  'date': '2020-06-25',
  'key': '1593117876348',
  'mon_year': 'Jun_2020',
  'note': 'Hynix RAM',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 4},
 {'amount': -29,
  'category_id': 38,
  'date': '2020-08-01',
  'key': '1596304675727',
  'mon_year': 'Aug_2020',
  'note': 'garena',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 4},
 {'amount': 400,


In [186]:
get_period('1592976346122')

{'amount': -149,
 'category_id': 11,
 'date': '2020-06-24',
 'key': '1592976346122',
 'mon_year': 'Jun_2020',
 'note': 'mou belt',
 'subcategory_id': 0,
 'trans_amount': 0,
 'transfer_wallet_id': -1,
 'type': 'exp',
 'wallet_id': 3}

In [131]:
dbfile = 'ignore/wallet-database.db'
con = sqlite3.connect(dbfile)
cur = con.cursor()
transaction_df = pd.read_sql_query('SELECT * FROM trans', con)          #? all transactions table
con.close()
df = transaction_df.copy()
df['amount'] = df['amount'].div(100).round(2)
df['trans_amount'] = df['trans_amount'].div(100).round(2)
df['type'] = df['type'].replace([0, 1, 2], ['inc', 'exp', 'tran'])
df.drop(['id', 'fee_id', 'account_id', 'debt_id', 'debt_trans_id', 'memo'], axis = 1, inplace = True)
df['period'] = df.date_time
df['date_time'] = pd.to_datetime(df["date_time"], unit='ms').dt.date
df = df[['period', 'date_time', 'type', 'amount', 'wallet_id', 'category_id', 'transfer_wallet_id', 'trans_amount', 'subcategory_id', 'note']]
df

,period,date_time,type,amount,wallet_id,category_id,transfer_wallet_id,trans_amount,subcategory_id,note
0,1598901225425,2020-08-31,exp,-50.0,1,30,-1,0.0,0,
1,1599032302625,2020-09-02,inc,25.0,2,34,-1,0.0,0,
2,1599068340714,2020-09-02,exp,-50.0,5,30,-1,0.0,0,
3,1599068417708,2020-09-02,exp,-49.0,2,32,-1,0.0,0,mama chhoto
4,1599068486385,2020-09-02,inc,50.0,1,33,-1,0.0,0,mama chhoto
...,...,...,...,...,...,...,...,...,...,...
2249,1686117556971,2023-06-07,exp,-24.0,11,6,-1,0.0,0,canteen
2250,1686228395238,2023-06-08,exp,-146.0,11,11,-1,0.0,0,me+dk - ezee35+basmatiRice65+aloo10+haldi36
2251,1686291222171,2023-06-09,exp,-32.0,11,6,-1,0.0,0,canteen
2252,1686402353190,2023-06-10,exp,-20.0,11,6,-1,0.0,0,me+dk-fuchka


In [137]:
# df.date_time = pd.to_datetime(df.date_time)
df.dtypes

period                         int64
date_time             datetime64[ns]
type                          object
amount                       float64
wallet_id                      int64
category_id                    int64
transfer_wallet_id             int64
trans_amount                 float64
subcategory_id                 int64
note                          object
year                          object
month                         object
mon_year                      object
dtype: object

In [135]:
df['year'] = df.date_time.dt.strftime('%Y')
df['month'] = df.date_time.dt.strftime('%b')
df['mon_year'] = df[['month', 'year']].agg('_'.join, axis=1)
df.mon_year

0       Aug_2020
1       Sep_2020
2       Sep_2020
3       Sep_2020
4       Sep_2020
          ...   
2249    Jun_2023
2250    Jun_2023
2251    Jun_2023
2252    Jun_2023
2253    Jun_2023
Name: mon_year, Length: 2254, dtype: object

In [155]:
df.shape

(2254, 13)

In [171]:
def insert_trans_from_df(df):
    """Inserts daily transaction"""
    for i in range(df.shape[0]):
        period = df.period.astype(str).iloc[i]
        date = df.date_time.astype(str).iloc[i]
        _type = df.type.iloc[i]
        amount = df.amount.astype(float).iloc[i]
        wallet_id = df.wallet_id.astype(float).iloc[i]
        category_id = df.category_id.astype(float).iloc[i]
        transfer_wallet_id = df.transfer_wallet_id.astype(float).iloc[i]
        trans_amount = df.trans_amount.astype(float).iloc[i]
        subcategory_id = df.subcategory_id.astype(float).iloc[i]
        note = df.note.iloc[i]
        mon_year = df.mon_year.iloc[i]
        print({'key': period, 'date': date, 'type': _type, 'amount': amount, 'wallet_id': wallet_id, 'category_id': category_id, 'transfer_wallet_id': transfer_wallet_id, 'trans_amount': trans_amount, 'subcategory_id': subcategory_id, 'note': note, 'mon_year':mon_year})
        db2.put({'key': period, 'date': date, 'type': _type, 'amount': amount, 'wallet_id': wallet_id, 'category_id': category_id, 'transfer_wallet_id': transfer_wallet_id, 'trans_amount': trans_amount, 'subcategory_id': subcategory_id, 'note': note, 'mon_year':mon_year})

In [172]:
insert_trans_from_df(df)

{'key': '1598901225425', 'date': '2020-08-31', 'type': 'exp', 'amount': -50.0, 'wallet_id': 1.0, 'category_id': 30.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': '', 'mon_year': 'Aug_2020'}
{'key': '1599032302625', 'date': '2020-09-02', 'type': 'inc', 'amount': 25.0, 'wallet_id': 2.0, 'category_id': 34.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': '', 'mon_year': 'Sep_2020'}
{'key': '1599068340714', 'date': '2020-09-02', 'type': 'exp', 'amount': -50.0, 'wallet_id': 5.0, 'category_id': 30.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': '', 'mon_year': 'Sep_2020'}
{'key': '1599068417708', 'date': '2020-09-02', 'type': 'exp', 'amount': -49.0, 'wallet_id': 2.0, 'category_id': 32.0, 'transfer_wallet_id': -1.0, 'trans_amount': 0.0, 'subcategory_id': 0.0, 'note': 'mama chhoto', 'mon_year': 'Sep_2020'}
{'key': '1599068486385', 'date': '2020-09-02', 'type': 'inc', 'amount': 50.0, 'wallet_

In [191]:
wallet_json = fetch_all_trans()
wallet_json

[{'amount': -149,
  'category_id': 11,
  'date': '2020-06-24',
  'key': '1592976346122',
  'mon_year': 'Jun_2020',
  'note': 'mou belt',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 3},
 {'amount': -299,
  'category_id': 11,
  'date': '2020-06-24',
  'key': '1592976705246',
  'mon_year': 'Jun_2020',
  'note': 'wedding shoe',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 3},
 {'amount': -1500,
  'category_id': 11,
  'date': '2020-06-25',
  'key': '1593117876348',
  'mon_year': 'Jun_2020',
  'note': 'Hynix RAM',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 4},
 {'amount': -29,
  'category_id': 38,
  'date': '2020-08-01',
  'key': '1596304675727',
  'mon_year': 'Aug_2020',
  'note': 'garena',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'exp',
  'wallet_id': 4},
 {'amount': 400,


In [221]:
df_new = pd.DataFrame(wallet_json)
df_new

,amount,category_id,date,key,mon_year,note,subcategory_id,trans_amount,transfer_wallet_id,type,wallet_id
0,-149.0,11,2020-06-24,1592976346122,Jun_2020,mou belt,0,0,-1,exp,3
1,-299.0,11,2020-06-24,1592976705246,Jun_2020,wedding shoe,0,0,-1,exp,3
2,-1500.0,11,2020-06-25,1593117876348,Jun_2020,Hynix RAM,0,0,-1,exp,4
3,-29.0,38,2020-08-01,1596304675727,Aug_2020,garena,0,0,-1,exp,4
4,400.0,0,2020-08-02,1596391075727,Aug_2020,,0,400,3,tran,2
...,...,...,...,...,...,...,...,...,...,...,...
995,-170.0,11,2021-09-26,1632664523061,Sep_2021,me+vik - 50banana+50maggi+40bread+30naphtha,0,0,-1,exp,2
996,-1001.0,1,2021-09-27,1632762692572,Sep_2021,policy payment,0,0,-1,exp,2
997,-2.0,14,2021-09-27,1632762877869,Sep_2021,policy mandate,0,0,-1,exp,2
998,-40.0,6,2021-09-29,1632896543145,Sep_2021,canteen,0,0,-1,exp,9


In [222]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   amount              1000 non-null   float64
 1   category_id         1000 non-null   int64  
 2   date                1000 non-null   object 
 3   key                 1000 non-null   object 
 4   mon_year            1000 non-null   object 
 5   note                1000 non-null   object 
 6   subcategory_id      1000 non-null   int64  
 7   trans_amount        1000 non-null   int64  
 8   transfer_wallet_id  1000 non-null   int64  
 9   type                1000 non-null   object 
 10  wallet_id           1000 non-null   int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 86.1+ KB
